In [3]:
# Import the relevant libraries
import ee
import geemap
import pandas as pd
# Initalize Earth Engine
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()


In [4]:
texas = ee.FeatureCollection("FAO/GAUL/2015/level2").filter(ee.Filter.eq('ADM1_NAME', 'Texas'))

# Access the solar potential raster image saved as a GEE asset
solarpot = ee.Image("users/ayanman18/pvpot")

# Add reducer output to the Features in the collection
solarPotFC = solarpot.reduceRegions(
    collection=texas,
    reducer=ee.Reducer.mean(),
    scale=1000  # Define an appropriate scale
)

# Get the result as a list of dictionaries
solarPotList = solarPotFC.getInfo()['features']

# Extract relevant properties into a Pandas DataFrame
solarPotdf = pd.DataFrame([{
    'ADM1_NAME': feature['properties']['ADM1_NAME'],
    'mean': feature['properties']['mean'],
    'ADM2_NAME': feature['properties']['ADM2_NAME']
} for feature in solarPotList])

# Sort the DataFrame by PV potential per unit area
solarPotdf_sorted = solarPotdf.sort_values(by='mean', ascending=False)

# Print the sorted DataFrame
print(solarPotdf_sorted)

    ADM1_NAME         mean   ADM2_NAME
69      Texas  1970.383587     El Paso
114     Texas  1957.271342    Hudspeth
54      Texas  1902.100474   Culberson
188     Texas  1886.894876    Presidio
21      Texas  1867.212874    Brewster
..        ...          ...         ...
78      Texas  1502.575443   Fort Bend
87      Texas  1500.922067      Goliad
169     Texas  1498.334691  Montgomery
234     Texas  1497.992881    Victoria
99      Texas  1492.458752      Hardin

[254 rows x 3 columns]


In [5]:
solarPotdf_sorted['mean'] = solarPotdf_sorted['mean']/365

In [6]:
solarPotdf_sorted.to_csv('solar_potential_mean.csv', index=False)